# Introducción a NLP con spaCy

> _Preparado por: [Juan Javier Santos Ochoa](https://twitter.com/jjsantoso) ([LNPP](https://www.lnpp.mx/))_

<img src="imagenes/SpaCy_logo.png" width=600>

* [spaCy es una librería](https://spacy.io/) de Python para hacer procesamiento del lenguaje natural (NLP), especialmente pensada para ser usada en producción.
* Es relativamente nueva y se ha hecho muy popular porque ya incluye modelos preentrenados para desarrollar algunas tareas de NLP, como la representación vectorial de palabras y el reconocimiento de partes del discurso y de entidades.
* spacy permite descargar modelos de diferentes idiomas, entre ellos el español. Estos modelos están construidos con técnicas de redes neuronales pero no necesitamos saber nada de redes neuronales para utilizarlos.
* Para instalar la librería podemos usar pip desde la terminal de comandos:
```
pip install spacy
```
* La versión más reciente es la 2.2.3 que es con la que trabajaremos a continuación.

In [ ]:
#!pip install spacy

In [ ]:
import spacy
from spacy import displacy
import pandas as pd
pd.options.display.max_colwidth = 300
print(pd.__name__, pd.__version__)
print(spacy.__name__, spacy.__version__)

* Podemos descargar los modelos de spacy también desde la terminal de comandos. Todos los modelos disponibles se [pueden consultar aquí](https://spacy.io/models)
* En este caso descargamos el modelo de idioma español, entrenado con un corpus de artículos de noticias.

In [ ]:
!python -m spacy download es_core_news_md

## Datos

* Usaremos un conjunto de datos que contiene noticias sobre corrupción en varios medios de México

In [ ]:
noticias = pd.read_csv('datos/noticias_corrupcion.csv')
noticias.head()

## Modelo
* Para cargar un modelo usamos la función `spacy.load()`
* Esto crea un objeto tipo lenguaje.
* Si le pasamos un texto lo convierte en un documento.

In [ ]:
nlp = spacy.load("es_core_news_md")
nlp

## Tipos de objetos

* Los principales objetos con los que trabaja spacy son los documentos, pero también son muy útiles los tokens y spans.

<img src="imagenes/span.png" width=600>

In [ ]:
doc = nlp("Esta es una frase de ejemplo")
doc

In [ ]:
type(doc)

* Un documento está compuesto de tokens y se puede indexar para obtener tokens individuales. EL resultado es un objeto tipo `Token`

In [ ]:
doc[1], type(doc[1])

* También se pueden indexar varios valores, como en una lista, y el resultado es un objeto tìpo `Span`

In [ ]:
(doc[2:4], type(doc[2:4]))

## Propiedades de los tokens
* Los tokens tienen algunos atributos y métodos útiles para el preprocesamiento de los datos. Algunos de esto son `.pos_`, `.lemma_`, `.is_stop`, `.tag_`

In [ ]:
for w in doc:
    print(w.text, w.pos_, w.lemma_, w.is_stop, w.tag_, sep=' | ')

> Puede consultar las stopwords de spacy en `spacy.lang.es.STOP_WORDS`

## Propiedades de los documentos

* Creamos un documento al pasar texto al modelo `nlp` que cargamos


In [ ]:
doc1 = nlp(noticias.loc[0, 'texto'])
doc2 = nlp(noticias.loc[10, 'texto'])

In [ ]:
doc1

### Representación vectorial
* Cada documento tiene una representación vectorial calculada a partir del modelo entrenado por spacy. El vector se puede ver con el atributo `.vector` del documento

In [ ]:
doc1.vector 

### Similitud
* Usando la representación vectorial podemos comparar varios documentos usando el método `.similarity()`
* El resultado es la similitud coseno, un coeficiente entre -1 y 1 que a mayor valor indica mayor similitud.

In [ ]:
doc1.similarity(doc2)

## Representación vectorial de palabras

* Spacy también incluye un vocabulario con las palabras que entrenó su modelo.
* Cada palabra también tiene una representación vectorial en el atributo `.vector`
* Con esta representación se puede calcular la similitud entre palabras o incluso entre palabras y documentos.

In [ ]:
vocab = nlp.vocab
lex = vocab['banco']
lex.vector

In [ ]:
lex2 = vocab['dinero']
lex.similarity(lex2)

In [ ]:
lex.similarity(doc1[0:10])

## Reconocimiento de entidades
* Los modelos de spacy tienen la capacidad de detectar entidades dentro del texto. Las entidades pueden ser personas, organizaciones, lugares, eventos, fechas, cantidades entre otros.
* En un documento automáticamente se identifican las entidades y el tipo de entidad que es. Estas se pueden encontrar en el atributo `.ents` de los documentos

In [ ]:
for e in doc1.ents:
    print(e.text, e.label_, spacy.explain(e.label_), sep='|')

In [ ]:
for e in doc2.ents:
    print(e.text, e.label_, spacy.explain(e.label_), sep='|')

* Spacy también incluye un módulo para visualizar las entidades encontradas. El módulo está en `spacy.displacy`
* Para visualizar dentro de un notebook se usa la función `.render()` sobre un documento, especificando las opciones `style="ent"` y `jupyter=True`

In [ ]:
displacy.render(doc1, style='ent', jupyter=True)

In [ ]:
displacy.render(doc2, style='ent', jupyter=True)

## Procesando varios documentos

* Si tenemos una colección de textos que queremos convertir a documentos de spacy, la manera más eficiente de hacerlo es usando `nlp.pipe`, que retorna un generador. Si queremos traerlos a memoria convertimos el resultado en una lista. Esta lista contendrá cada documento.

In [ ]:
docs_corrupcion = list(nlp.pipe(noticias['texto']))

In [ ]:
doc3 = docs_corrupcion[199]

In [ ]:
displacy.render(doc3, jupyter=True, style='ent')